<a href="https://colab.research.google.com/github/Svensone/fast.ai-coursev3/blob/master/%5Bfasti_ai%5D_Plant_Seedling_Image_Recognition_Kaggle_Plant_Seedling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Check if fastai and pytorch are installed
!pip list | grep -e fastai -e torch -e torchvision

fastai                   1.0.61         
torch                    1.5.0+cu101    
torchsummary             1.5.1          
torchtext                0.3.1          
torchvision              0.6.0+cu101    


In [0]:
# connect to gdrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# WATCH OUT when upload to KAGGLE or GITHUB ... API KEY !!!!!!!!!!

# Install Kaggle and register API Key

!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"","key":""}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
# create 'content' folder

import os
if os.getcwd() != '/content':
    os.chdir('/content')
    print("Directory changed.")

In [0]:
!kaggle datasets list -m

401 - Unauthorized


In [0]:
# download dataset and unzip

!kaggle datasets download -d vbookshelf/v2-plant-seedlings-dataset --force

import zipfile
with zipfile.ZipFile("v2-plant-seedlings-dataset.zip","r") as zip_ref:
    zip_ref.extractall("plant-seedlings-data")

401 - Unauthorized


FileNotFoundError: ignored

In [0]:

labels = os.listdir("plant-seedlings-data")
print("No. of labels: {}".format(len(labels)))
print("----------")

for label in labels:
  print("{}, {} files".format(label, len(os.listdir("plant-seedlings-data/"+label))))

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

fig, ax = plt.subplots(nrows= 3, ncols=4)
fig.tight_layout()
cnt = 0

for row in ax:
  for col in row:
    image_name = np.random.choice(os.listdir(data_path+"/"+labels[cnt]))
    im = Image.open('plant-seedlings-data/{}/{}'.format(labels[cnt], image_name))
    col.imshow(im)
    col.set_title(labels[cnt])
    col.axis('off')
    cnt += 1
plt.show()


In [0]:
## 3. Creating the model

from fastai.vision import *
from fastai.metrics import accuracy



In [0]:
# create Data ImageDataBunch

# transform Images
# parameters

path = './plant-seedlings-data/'
bs = 64
size = 224 # image-size

data = ImageDataBunch.from_folder(path, 
                                  ds_tfms=get_transforms(do_flip=True, flip_vert=True),
                                  valid_pct=0.2, size=size, bs=bs)

In [0]:
# normalize data (imagenet, cifar and mnist possible)
# since later resnet trained on ImageNet normalization on ImageNet

data.normalize(imagenet_stats)

In [0]:
# create Model

learner = create_cnn(data, 
                     models.resnet18, 
                     metrics=[accuracy], 
                     callback_fns=ShowGraph)

In [0]:
learner.lr_find()
learner.recorder.plot()

In [0]:
learner.fit_one_cycle(8, max_lr= slice(1e-3, 1e-2))

In [0]:
# Interpret Results

interpreter = ClassificationInterpretation.from_learner(learner)

interpreter.most_confused(min_val=2)

In [0]:
## Improving the Model

### Unfreeze the model and fine-tune

In [0]:
# save weights and unfreeze

model.save('stage-1')
#model.load('stage-1')

learner.unfreeze()

learner.fit_one_cycle(2, max_lr = 1e-5, 1e-4)